In [1]:
# Necessary imports
import sys
import warnings
import pandas as pd
import numpy as np

sys.path.append('../../..')
from seismiqb.batchflow.research import Results

warnings.filterwarnings("ignore")

In [2]:
res_name = 'Research_horizons'
results = Results(res_name)
len(results.df)

252

In [3]:
df = results.df

In [4]:
df.columns

Index(['name', 'coverages', 'window_rates', 'corrs', 'phases',
       'avg_l1_in_holes', 'iteration', 'sample_index', 'cube_and_horizon',
       'seed', 'crop_shape', 'repetition', 'update'],
      dtype='object')

In [5]:
df['cube_name'] = df['cube_and_horizon'].apply(lambda item: item.split('+')[0])
df['horizon_name'] = df['cube_and_horizon'].apply(lambda item: item.split('+')[1])
df = df.drop(columns=['name', 'cube_and_horizon', 'sample_index', 'update'])

In [6]:
groupby_index = ['cube_name', 'horizon_name', 'seed']

In [7]:
df = df.reset_index(drop=True).set_index(groupby_index)

In [8]:
research_config_cols = ['iteration', 'crop_shape', 'repetition']

In [9]:
wr = df['window_rates'].apply(pd.Series).rename({i : f'window_rate_{i}' for i in range(3)}, axis=1)
cov = df['coverages'].apply(pd.Series).rename({i : f'coverage_{i}' for i in range(3)}, axis=1)
corrs = df['corrs'].apply(pd.Series).rename({i : f'corr_{i}' for i in range(3)}, axis=1)
phases = df['phases'].apply(pd.Series).rename({i : f'phases_{i}' for i in range(3)}, axis=1)
avg_l1_in_holes = df['avg_l1_in_holes'].apply(pd.Series).rename({i : f'avg_l1_in_holes_{i}' for i in range(3)}, axis=1)

df = pd.concat((wr, cov, corrs, phases, avg_l1_in_holes, df[research_config_cols]), axis=1).sort_index()

In [11]:
last_iter_df = df.loc[(df.iteration == df.iteration.max())] # & (df.repetition == df.repetition.max())]

In [12]:
metric = 'corr_1'
grpby_last_iter_df = last_iter_df.groupby(groupby_index)[metric].max()
merged = last_iter_df.merge(grpby_last_iter_df)

In [13]:
merged.index = grpby_last_iter_df.index

In [20]:
np.unique(merged['crop_shape'], return_counts=True)

(array(['(1, 128, 128)', '(1, 64, 64)'], dtype=object), array([27, 36]))

In [21]:
np.unique(merged['repetition'], return_counts=True)

(array([0, 1]), array([34, 29]))

In [22]:
np.unique(merged['iteration'], return_counts=True)

(array([0]), array([63]))